In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import json
import string
import pickle
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt

### **Given Code with Forward() Filled**

In [2]:
# Define GitHub file URLs for data download
train_data_url = "https://raw.githubusercontent.com/saivikas10/Assignment-2/refs/heads/main/training.json"
validation_data_url = "https://raw.githubusercontent.com/saivikas10/Assignment-2/refs/heads/main/validation.json"
test_data_url = "https://raw.githubusercontent.com/saivikas10/Assignment-2/refs/heads/main/test.json"


In [3]:
# Download data files from GitHub
def download_data(url, filename):
    response = requests.get(url)
    with open(filename, 'w') as file:
        file.write(response.text)

download_data(train_data_url, 'training.json')
download_data(validation_data_url, 'validation.json')
download_data(test_data_url, 'test.json')
!wget "https://media.githubusercontent.com/media/saivikas10/Assignment-2/refs/heads/main/word_embedding.pkl" -O "word_embedding.pkl"
print("Data files downloaded successfully.")


--2024-11-12 17:42:31--  https://media.githubusercontent.com/media/saivikas10/Assignment-2/refs/heads/main/word_embedding.pkl
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108556651 (104M) [application/octet-stream]
Saving to: ‘word_embedding.pkl’

word_embedding.pkl  100%[===================>] 103.53M  76.9MB/s    in 1.3s    

2024-11-12 17:42:34 (76.9 MB/s) - ‘word_embedding.pkl’ saved [108556651/108556651]

Data files downloaded successfully.


In [4]:
# Define the RNN model class
class RNN(nn.Module):
  def __init__(self, input_dim, h):  # Add relevant parameters
        super(RNN, self).__init__()
        self.h = h
        self.numOfLayer = 1
        self.rnn = nn.RNN(input_dim, h, self.numOfLayer, nonlinearity='tanh', batch_first=True)
        self.W = nn.Linear(h, 5)
        self.softmax = nn.LogSoftmax(dim=1)
        self.loss = nn.NLLLoss()

  def compute_Loss(self, predicted_vector, gold_label):
        return self.loss(predicted_vector, gold_label)

  def forward(self, inputs):
        # Obtain hidden layer representation
        _, hidden = self.rnn(inputs)
        # Obtain output layer representations
        out = self.W(hidden[:, -1, :])
        # Obtain probability distribution
        predicted_vector = self.softmax(out)
        return predicted_vector

In [5]:

unk = '<UNK>'


def load_data(train_data, val_data):
  with open(train_data) as training_f:
      training = json.load(training_f)
  with open(val_data) as valid_f:
      validation = json.load(valid_f)

  tra = []
  val = []
  for elt in training:
      tra.append((elt["text"].split(), int(elt["stars"] - 1)))
  for elt in validation:
      val.append((elt["text"].split(), int(elt["stars"] - 1)))
  return tra, val

# Load word embeddings
def load_embeddings(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

# Load data
print("========== Loading data ==========")
training_path = 'training.json'
validation_path = 'validation.json'
training_data, validation_data = load_data(training_path,  validation_path)  # Load data
print("Data loaded successfully.")

# Prepare word embeddings
print("========== Vectorizing data ==========")
word_embedding = load_embeddings('word_embedding.pkl')  # Make sure to upload this file

# Set default configurations for Colab
hidden_dim = 10
epochs = 5
minibatch_size = 16

print("==========Running for Default Configuration==========")
print("Hidden_dim =", hidden_dim)
print("Epochs =", epochs)
print("Learning Rate =", 0.01)
print("Batch Size =", minibatch_size)

model = RNN(50, hidden_dim)  # Initialize model
optimizer = optim.Adam(model.parameters(), lr=0.01)

stopping_condition = False
epoch = 0
last_train_accuracy = 0
last_validation_accuracy = 0

while not stopping_condition:
    random.shuffle(training_data)
    model.train()
    print("Training started for epoch {}".format(epoch + 1))
    correct = 0
    total = 0
    N = len(training_data)

    loss_total = 0
    loss_count = 0
    for minibatch_index in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for example_index in range(minibatch_size):
            input_words, gold_label = training_data[minibatch_index * minibatch_size + example_index]
            input_words = " ".join(input_words)

            # Remove punctuation
            input_words = input_words.translate(input_words.maketrans("", "", string.punctuation)).split()

            # Look up word embedding dictionary
            vectors = [word_embedding.get(i.lower(), np.zeros(next(iter(word_embedding.values())).shape)) for i in input_words]

            # Transform the input into required shape and convert to float32
            vectors = torch.tensor(vectors, dtype=torch.float32).view(len(vectors), 1, -1)  # Shape to (seq_len, 1, embedding_dim)
            output = model(vectors)

            # Get loss
            example_loss = model.compute_Loss(output.view(1, -1), torch.tensor([gold_label]))

            # Get predicted label
            predicted_label = torch.argmax(output)

            correct += int(predicted_label == gold_label)
            total += 1
            if loss is None:
                loss = example_loss
            else:
                loss += example_loss

        loss = loss / minibatch_size
        loss_total += loss.data
        loss_count += 1
        loss.backward()
        optimizer.step()

    avg_loss = loss_total / loss_count
    print("Training completed for epoch {}".format(epoch + 1))
    print("Training accuracy for epoch {}: {}".format(epoch + 1, correct / total))
    print(f"Average Loss: {avg_loss:.4f}")
    training_accuracy = correct / total

    # Validation
    model.eval()
    correct = 0
    total = 0
    random.shuffle(validation_data)
    print("Validation started for epoch {}".format(epoch + 1))

    for input_words, gold_label in tqdm(validation_data):
        input_words = " ".join(input_words)
        input_words = input_words.translate(input_words.maketrans("", "", string.punctuation)).split()
        vectors = [word_embedding.get(i.lower(), np.zeros(next(iter(word_embedding.values())).shape)) for i in input_words]

        vectors = torch.tensor(vectors, dtype=torch.float32).view(len(vectors), 1, -1)  # Ensure the correct shape
        output = model(vectors)
        predicted_label = torch.argmax(output)
        correct += int(predicted_label == gold_label)
        total += 1

    print("Validation completed for epoch {}".format(epoch + 1))
    print("Validation accuracy for epoch {}: {}".format(epoch + 1, correct / total))
    validation_accuracy = correct / total

    if validation_accuracy < last_validation_accuracy and training_accuracy > last_train_accuracy:
        stopping_condition = True
        print("Training done to avoid overfitting!")
        print("Best validation accuracy is:", last_validation_accuracy)
    else:
        last_validation_accuracy = validation_accuracy
        last_train_accuracy = training_accuracy

    epoch += 1

print("Final Validation Accuracy:", last_validation_accuracy)


========== Loading data ==========
Data loaded successfully.
========== Vectorizing data ==========
==========Running for Default Configuration==========
Hidden_dim = 10
Epochs = 5
Learning Rate = 0.01
Batch Size = 16
Training started for epoch 1


  0%|          | 0/1000 [00:00<?, ?it/s]<ipython-input-5-699f77545af5>:77: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  vectors = torch.tensor(vectors, dtype=torch.float32).view(len(vectors), 1, -1)  # Shape to (seq_len, 1, embedding_dim)
100%|██████████| 1000/1000 [00:58<00:00, 16.96it/s]


Training completed for epoch 1
Training accuracy for epoch 1: 0.2460625
Average Loss: 1.5953
Validation started for epoch 1


100%|██████████| 800/800 [00:01<00:00, 437.34it/s]


Validation completed for epoch 1
Validation accuracy for epoch 1: 0.2575
Training started for epoch 2


...

Validation completed for epoch 2
Validation accuracy for epoch 2: 0.3
Training started for epoch 3


100%|██████████| 1000/1000 [00:45<00:00, 21.92it/s]


Training completed for epoch 3
Training accuracy for epoch 3: 0.2643125
Average Loss: 1.5747
Validation started for epoch 3


100%|██████████| 800/800 [00:01<00:00, 462.58it/s]

Validation completed for epoch 3
Validation accuracy for epoch 3: 0.2475
Training done to avoid overfitting!
Best validation accuracy is: 0.3
Final Validation Accuracy: 0.3


### **Code to Run Different Configurations and Plot the Best Model**

In [ ]:
# Set configurations for running the RNN
configs = [
    {'epochs': 5, 'hidden_dim': 5, 'learning_rate': 0.01, 'batch_size': 16},
    {'epochs': 10, 'hidden_dim': 5, 'learning_rate': 0.01, 'batch_size': 16},
    {'epochs': 5, 'hidden_dim': 10, 'learning_rate': 0.1, 'batch_size': 32},
    {'epochs': 10, 'hidden_dim': 10, 'learning_rate': 0.01, 'batch_size': 32},
    {'epochs': 15, 'hidden_dim': 15, 'learning_rate': 0.001, 'batch_size': 64}
]

best_validation_accuracy = 0
best_train_losses, best_train_accuracies, best_validation_accuracies = [], [], []
best_configuration = None

# Run each configuration
for config_num, config in enumerate(configs, start=1):

    print(f"\nCONFIGURATION {config_num}:")
    hidden_dim = config['hidden_dim']
    epochs = config['epochs']
    learning_rate = config['learning_rate']
    batch_size = config['batch_size']

    print(f"Hidden Dimension: {hidden_dim}, Epochs: {epochs}, Learning Rate: {learning_rate}, Batch Size: {batch_size}")

    # Initializing model and optimizer
    model = RNN(50, hidden_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses, train_accuracies, val_accuracies = [], [], []

    for epoch in range(epochs):
        correct_train = 0
        total_train = 0
        total_loss = 0

        print(f"Epoch {epoch + 1}/{epochs}:")

        # Training
        model.train()
        for minibatch_index in tqdm(range(len(training_data) // batch_size), desc=f"Epoch {epoch + 1} Progress"):
            optimizer.zero_grad()
            batch_loss = None
            for example_index in range(batch_size):
                input_words, gold_label = training_data[minibatch_index * batch_size + example_index]
                input_words = " ".join(input_words).translate(str.maketrans("", "", string.punctuation)).split()
                vectors = [word_embedding.get(word.lower(), np.zeros(50)) for word in input_words]
                vectors = torch.tensor(vectors, dtype=torch.float32).view(len(vectors), 1, -1)
                output = model(vectors)
                example_loss = model.compute_Loss(output.view(1, -1), torch.tensor([gold_label]))

                if batch_loss is None:
                    batch_loss = example_loss
                else:
                    batch_loss += example_loss

                predicted_label = torch.argmax(output)
                correct_train += int(predicted_label == gold_label)
                total_train += 1

            batch_loss.backward()
            optimizer.step()
            total_loss += batch_loss.item()

        avgerage_train_loss = total_loss / (len(training_data) // batch_size)
        train_accuracy = correct_train / total_train
        train_losses.append(avgerage_train_loss)
        train_accuracies.append(train_accuracy)

        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for input_words, gold_label in validation_data:
                input_words = " ".join(input_words).translate(str.maketrans("", "", string.punctuation)).split()
                vectors = [word_embedding.get(word.lower(), np.zeros(50)) for word in input_words]
                vectors = torch.tensor(vectors, dtype=torch.float32).view(len(vectors), 1, -1)
                output = model(vectors)
                predicted_label = torch.argmax(output)
                correct_val += int(predicted_label == gold_label)
                total_val += 1

        val_accuracy = correct_val / total_val
        val_accuracies.append(val_accuracy)

        # Print epoch results in Code 2 format
        print(f"Training Loss = {avgerage_train_loss:.4f}, Training Accuracy = {train_accuracy:.4f}, Validation Accuracy = {val_accuracy:.4f}")
    # Print final training and validation accuracy for this configuration
    print(f"\nFinal Training Accuracy for Config {config_num}: {train_accuracy:.4f}")
    print(f"Final Validation Accuracy for Config {config_num}: {val_accuracy:.4f}")
    print("==============\n")

    # Checking if this configuration has the best validation accuracy
    if val_accuracies[-1] > best_validation_accuracy:
        best_validation_accuracy = val_accuracies[-1]
        best_train_losses = train_losses
        best_train_accuracies = train_accuracies
        best_validation_accuracies = val_accuracies
        best_configuration = config

# Print the best configuration and validation accuracy
print("\nBest Configuration:")
print(f"Hidden Dim: {best_configuration['hidden_dim']}")
print(f"Epochs: {best_configuration['epochs']}")
print(f"Learning Rate: {best_configuration['learning_rate']}")
print(f"Batch Size: {best_configuration['batch_size']}")
print(f"Best Validation Accuracy: {best_validation_accuracy:.4f}")


CONFIGURATION 1:
Hidden Dimension: 5, Epochs: 5, Learning Rate: 0.01, Batch Size: 16
Epoch 1/5:


Epoch 1 Progress: 100%|██████████| 1000/1000 [00:44<00:00, 22.66it/s]


Training Loss = 25.5340, Training Accuracy = 0.2366, Validation Accuracy = 0.2637
Epoch 2/5:


Epoch 2 Progress: 100%|██████████| 1000/1000 [00:49<00:00, 20.27it/s]


Training Loss = 25.3288, Training Accuracy = 0.2539, Validation Accuracy = 0.2737
Epoch 3/5:


...

Epoch 7 Progress: 100%|██████████| 250/250 [00:42<00:00,  5.85it/s]


Training Loss = 100.4116, Training Accuracy = 0.2745, Validation Accuracy = 0.2450
Epoch 8/15:


Epoch 8 Progress: 100%|██████████| 250/250 [00:41<00:00,  5.99it/s]


Training Loss = 100.2921, Training Accuracy = 0.2764, Validation Accuracy = 0.2475
Epoch 9/15:


Epoch 9 Progress:   4%|▍         | 10/250 [00:01<00:41,  5.84it/s]

In [ ]:
# Plot Training Loss, Training Accuracy, and Validation Accuracy for the best model
plt.figure(figsize=(18, 5))

# Training Loss plot
plt.subplot(1, 3, 1)
plt.title('Training Loss vs Epochs (Best Model)')
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.plot(range(1, len(best_train_losses) + 1), best_train_losses, marker='o')

# Training Accuracy plot
plt.subplot(1, 3, 2)
plt.title('Training Accuracy vs Epochs (Best Model)')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.plot(range(1, len(best_train_accuracies) + 1), best_train_accuracies, marker='o')

# Validation Accuracy plot
plt.subplot(1, 3, 3)
plt.title('Validation Accuracy vs Epochs (Best Model)')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.plot(range(1, len(best_validation_accuracies) + 1), best_validation_accuracies, marker='o')

plt.tight_layout()
plt.show()

### **Testing the Best Model**

In [9]:
# Loading test data
test_data_path = 'test.json'
test_data, _ = load_data(test_data_path, test_data_path)

# Initializing the best model according on the best configuration
BestModel = RNN(50, best_configuration['hidden_dim'])
BestModel.eval()  # Setting model to evaluation mode

test_correct = 0
test_total = 0

# Calculating test accuracy
with torch.no_grad():
    for input_words, gold_label in test_data:
        # Prepare the input by removing punctuation and converting to embeddings
        input_words = " ".join(input_words).translate(str.maketrans("", "", string.punctuation)).split()
        vectors = [word_embedding.get(word.lower(), np.zeros(50)) for word in input_words]
        vectors = torch.tensor(vectors, dtype=torch.float32).view(len(vectors), 1, -1)

        #Getting model output and calculating accuracy
        output = BestModel(vectors)
        predicted_label = torch.argmax(output)
        test_correct += int(predicted_label == gold_label)
        test_total += 1

# Calculating test accuracy
Testing_Accuracy = test_correct / test_total if test_total > 0 else 0
print(f"\nTest Accuracy for Best Model: {Testing_Accuracy:.4f}")



Test Accuracy for Best Model: 0.3738
